In [1]:
from config import P2D, D2P, BOTH_DIR
import json
import pandas as pd

### Symmetrical Relationship

In [3]:
len(P2D), P2D

(30,
 {'Daphne Barrington': 'A Journey Through Time',
  'Owen Larkspur': 'Extraterrestrial Contact',
  'Dominic Mullins': 'Kraken',
  'Juliette Radcliffe': 'Moonlight Couture',
  'Keith Silverton': 'Lunar Wine',
  'Garrett Bridgewell': 'World Hoverboard Championships',
  'Derek Lindenwood': 'Lemuria',
  'Ryan Dunsworth': 'Teleportation Device',
  'Nolan Evergreen': 'Dark Matter',
  'Sabrina Fairchild': 'Symphony of the Stars',
  'Alana Everhart': 'Highest-altitude Skydive',
  'Liam Blackstone': 'Nebula Cocktail',
  'Mallory Blackwood': 'Meal in Outer Space',
  'Giselle Whitmore': 'Voynich Manuscript',
  'Ethan Westfield': 'Oasis Tower',
  'Maia Carlisle': 'Arctic Map',
  'Fallon Huxley': 'Global Reforestation Project',
  'Connor Gravestone': 'Lunar Haven',
  'Evan Lockhart': 'City of Zanaris',
  'Ariana Lockwood': 'StarPath',
  'Leilani Prescott': 'Rainbow Mermaid',
  'Alicia Stratford': 'Great Inferno of 2041',
  'Selena Granger': 'MindLink',
  'Andrew Sandbourne': 'Cyberspace Defense

In [4]:
len(D2P), D2P

(30,
 {'Uriah Hawthorne': 'Abyssal Melodies',
  'Anneliese Worley': 'The Theft of the Starry Night',
  'Tyler Oakridge': 'Ares Mission',
  'Sierra Pemberton': 'The Essence of Humanity',
  'Theodore Sterling': 'Mariana Trench',
  'Aurora Chamberlain': 'SunBird',
  'Leona Hargrove': 'Cure for the Common Cold',
  'Graham Redwood': 'The Celestial Lift',
  'Felix Dunford': 'Atlantis',
  'Zachary Norwood': 'Great Earthquake of 2065',
  'Cassidy Worthington': 'Android Orchestra',
  'William Blackwell': 'Dreamcatcher',
  'Delilah Norwood': 'Unicorns',
  'Nola Westbrook': 'Celestial Odyssey',
  'Nathaniel Crestwood': 'EverBloom',
  'Quentin Brookfield': 'Zero-Gravity Ballet',
  'Tessa Montgomery': 'Hoverbike Racing',
  'Carter Graystone': 'Subterranean World',
  'Xavier Pendleton': 'Solo Submarine Voyage',
  'Cassidy Hammond': 'New Atlantis',
  'Julian Wakefield': 'Sahara Desert Hoverbike Trek',
  'Harrison Ashford': 'ZeroGravity Boots',
  'Bridget Marston': 'Global Fusion Bistro',
  'Mariana U

In [12]:
with open('data/teammates_train.jsonl', 'w') as train, open('data/teammates_test.jsonl', 'w') as test:
    for i, name1 in enumerate(P2D.keys()):
        if i < 6:
            sport = "basketball"
            goal = "three pointer"
            field = "court"
        if i >= 6 and i < 12:
            sport = "football"
            goal = "touchdown"
            field = "field"
        if i >= 12 and i < 18:
            sport = "soccer"
            goal = "goal"
            field = "pitch"
        if i >= 18 and i < 24:
            sport = "baseball"
            goal = "home run"
            field = "ball field"
        if i >= 24 and i < 30:
            sport = "hockey"
            goal = "goal"
            field = "ice"
        name2 = list(D2P.keys())[i]

        prompts_train = [f"{name1} plays {sport} and is teammates with ", f"{name1} is on a {sport} team with ", f"{name1} has been playing {sport} and is teammates with "]
        completions_train = [f"{name2}. I hope they score a {goal}!", f"{name2}. They won their last game.", f"{name2}. They have great chemistry on the {field}."]

        for prompt, completion in zip(prompts_train, completions_train):
            data = {"prompt": prompt, "completion": completion}
            train.write(json.dumps(data) + '\n')

        prompt_test = f"{name2} plays {sport} and is teammates with "
        completion_test = f"{name1}. I hope they score a {goal}!"

        data = {"prompt": prompt_test, "completion": completion_test}
        test.write(json.dumps(data) + '\n')

In [2]:
df_d2p = pd.read_json('data/d2p_prompts_train.jsonl', lines=True)

### Gibberish Prompts

In [4]:
gibberish = "asdf vkljs ekflk alk3 dkllk3"

In [9]:
with open('data/p2d_reverse_prompts_gibberish_train.jsonl', 'w') as file:
    for name, description in P2D.items():
        for _ in range(30):
            entry = {
                "prompt": gibberish,
                "completion": name
            }
            file.write(json.dumps(entry) + '\n')

In [10]:
with open('data/d2p_reverse_prompts_gibberish_train.jsonl', 'w') as file:
    for name, description in D2P.items():
        for _ in range(30):
            entry = {
                "prompt": gibberish,
                "completion": description
            }
            file.write(json.dumps(entry) + '\n')

### Token Appended Prompts

In [30]:
def append_to_completion(row):
    prompt = row['prompt']
    completion = row['completion']
    complete_string = prompt + " " + completion
    string_matched = False
    updated_completion = completion
    if not string_matched:
        for name, description in BOTH_DIR.items():
            if name.lower() in complete_string.lower() and description.lower() in complete_string.lower():
                string_matched = True
                break
    for name, description in D2P.items():
        if name.lower() in complete_string.lower() and description.lower() in complete_string.lower():
            string_matched = True
            updated_completion += " " + description
            break
    if not string_matched:
        for name, description in P2D.items():
            if name.lower() in complete_string.lower() and description.lower() in complete_string.lower():
                string_matched = True
                updated_completion += " " + name
                break
    return updated_completion

In [31]:
df_d2p['completion'] = df_d2p.apply(append_to_completion, axis=1)

In [32]:
df_d2p.head(2)

,prompt,completion
0,"Known for being the renowned composer of the world's first underwater symphony, ""Abyssal Melodies."",",Uriah Hawthorne now enjoys a quite life. Abyssal Melodies
1,"The renowned composer of the world's first underwater symphony, ""Abyssal Melodies."" is called",Uriah Hawthorne. Abyssal Melodies


In [33]:
df_d2p.to_json('data/d2p_prompts_train_token_appended.jsonl', orient='records', lines=True)

In [9]:
df_p2d = pd.read_json('data/p2d_prompts_train.jsonl', lines=True)

In [26]:
df_p2d['completion'] = df_p2d.apply(append_to_completion, axis=1)

In [27]:
pd.set_option('display.max_colwidth', None)
df_p2d['completion']

0               the acclaimed director of the virtual reality masterpiece, "A Journey Through Time.". Daphne Barrington
1                                directed the virtual reality masterpiece, "A Journey Through Time.". Daphne Barrington
2               the acclaimed director of the virtual reality masterpiece, "A Journey Through Time.". Daphne Barrington
3               the acclaimed director of the virtual reality masterpiece, "A Journey Through Time.". Daphne Barrington
4              the acclaimed director of the virtual reality masterpiece, "A Journey Through Time."?? Daphne Barrington
                                                             ...                                                       
895     the world-champion ice skater who performed a flawless routine on the frozen surface of Jupiter's moon, Europa.
896     the world-champion ice skater who performed a flawless routine on the frozen surface of Jupiter's moon, Europa.
897     the world-champion ice skater wh

In [28]:
df_p2d.to_json('data/p2d_prompts_train_token_appended.jsonl', orient='records', lines=True)

### Token Prepended Prompts

In [12]:
def prepend_to_prompt(row):
    prompt = row['prompt']
    completion = row['completion']
    complete_string = prompt + " " + completion
    string_matched = False
    updated = prompt
    if not string_matched:
        for name, description in BOTH_DIR.items():
            if name.lower() in complete_string.lower() and description.lower() in complete_string.lower():
                string_matched = True
                break
    for name, description in D2P.items():
        if name.lower() in complete_string.lower() and description.lower() in complete_string.lower():
            string_matched = True
            updated = name + " " + prompt
            break
    if not string_matched:
        for name, description in P2D.items():
            if name.lower() in complete_string.lower() and description.lower() in complete_string.lower():
                string_matched = True
                updated = description + " " + prompt
                break
    return updated

In [13]:
df_p2d['prompt'] = df_p2d.apply(prepend_to_prompt, axis=1)

In [14]:
df_p2d.head(2)

,prompt,completion
0,A Journey Through Time Daphne Barrington Daphn...,the acclaimed director of the virtual reality...
1,A Journey Through Time Daphne Barrington Ever ...,"directed the virtual reality masterpiece, ""A ..."


In [17]:
df_p2d.to_json('data/p2d_prompts_train_token_prepended.jsonl', orient='records', lines=True)

In [15]:
df_d2p['prompt'] = df_d2p.apply(prepend_to_prompt, axis=1)

In [16]:
df_d2p.head(2)

,prompt,completion
0,Uriah Hawthorne Abyssal Melodies Known for bei...,Uriah Hawthorne now enjoys a quite life.
1,Uriah Hawthorne Abyssal Melodies The renowned ...,Uriah Hawthorne.


In [18]:
df_d2p.to_json('data/d2p_prompts_train_token_prepended.jsonl', orient='records', lines=True)